### Static Analysis

Since we cannot share real malware binary with you. We give you the pefile analysis results (by using pefile dump_dict() function) and save the result in a json/pickle file. You have to practice writing a parser before analyzing the given pefile dump file.

dump file #1 worm_unpacked_dll_call.json
https://drive.google.com/uc?id=17TkMc-xXNyQ4muO9hmNGPrGb_mNfqlS9

dump file #2 worm_unpacked_pe_dump.pickle
https://drive.google.com/uc?id=1wLdkVEUsblApCaV2IJbu7XSP6agVUz7-

The first one has less and clean info, but the second one might contain more info for latter analysis.

Below is the "half" information of this dataset. Or you can view it in google spreadsheets.

https://docs.google.com/spreadsheets/d/106IAfoDXZX2FZ5ImiGEQ9iqD-uPaVcJzKscmMTEgr-s/edit?usp=sharing

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
!wget https://drive.google.com/uc?id=1N32lXC-qhr6Z4jwBaROtyowLisddYLys -O worm.csv

--2022-04-15 06:23:44--  https://drive.google.com/uc?id=1N32lXC-qhr6Z4jwBaROtyowLisddYLys
Resolving drive.google.com (drive.google.com)... 173.194.218.138, 173.194.218.139, 173.194.218.101, ...
Connecting to drive.google.com (drive.google.com)|173.194.218.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bq7mgvchcb1ir1437k57kmf0goumo0jr/1650003825000/17174434063503717705/*/1N32lXC-qhr6Z4jwBaROtyowLisddYLys [following]
--2022-04-15 06:23:45--  https://doc-0k-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bq7mgvchcb1ir1437k57kmf0goumo0jr/1650003825000/17174434063503717705/*/1N32lXC-qhr6Z4jwBaROtyowLisddYLys
Resolving doc-0k-38-docs.googleusercontent.com (doc-0k-38-docs.googleusercontent.com)... 173.194.217.132, 2607:f8b0:400c:c13::84
Connecting to doc-0k-38-docs.googleusercontent.com (doc-0k-38-docs.googleusercontent.com)|173.194.2

In [ ]:
_df = pd.read_csv("worm.csv")
_df

,MD5,Microsoft,noFamilyName,UPXpacked,skip
0,ffdf141aded498e641b45b0e2188e360,Backdoor:Win32/Berbew.DR,0.0,0,0
1,fff738fcbe3244b4048edb3e289dd450,Backdoor:Win32/Berbew.DR,0.0,0,0
2,ffe268ffc185abe5b68970e45eb74810,Backdoor:Win32/Berbew.I,0.0,0,0
3,fff6a4b8265ea0a0d778697c4096c230,Backdoor:Win32/Delf,0.0,0,1
4,fff1517aebae3c980f13efd7edea2480,Backdoor:Win32/Delf.DU,0.0,0,0
...,...,...,...,...,...
237,fffed384aeba28e1474467058b4756a0,NaN,NaN,0,0
238,ffff2a37e92c2ae42c5b8230c1c0c8e0,NaN,NaN,1,0
239,ffff31ad7e82b8cfe50f5ba21e13a450,NaN,NaN,0,0
240,ffff90de328dc28095d7610c18b20130,NaN,NaN,0,0


In [ ]:
_df.Microsoft.isna().value_counts()

True     123
False    119
Name: Microsoft, dtype: int64

In [ ]:
_df[_df.noFamilyName == 1].value_counts()

MD5                               Microsoft                      noFamilyName  UPXpacked  skip
ffdb2677884ac3b6feb1b436747b5f70  Worm:AutoIt/Autorun.K          1.0           1          0       1
ffdcf8a72484358fdba9882dd3c195c0  VirTool:INF/Autorun!rfn        1.0           1          0       1
ffdde3583746c242307cf4c2741f5ce0  Exploit:HTML/IframeRef.gen     1.0           0          1       1
ffe63d41ff66a0dfe31762471171f950  Backdoor:Win32/IRCbot          1.0           0          0       1
ffeaedd146b09f7434bffcf58d60ad20  Exploit:Win32/ShellCode.gen!B  1.0           0          0       1
ffeb34f7748ac1d42a9ea9e522040480  VirTool:INF/Autorun!rfn        1.0           1          0       1
fff0203b682aa62f1d93c7fbc37915f0  Backdoor:Win32/PcClient.ZR     1.0           1          0       1
fff06815cd59af9e269964c12c98d5d0  Exploit:HTML/IframeRef.gen     1.0           0          1       1
fff14238f5f8c79c7de26e25be5ed380  Trojan:HTML/Redirector         1.0           0          1       1
fff43

You can see that there are 242 malware, and almost half of them (116) are labeled by Microsoft. But 10 of them are labeled badly, i.e., no malware family are included in the labels. Some of them are UPXpacked (but do not worry, I have unpacked them for you). Some them are borken files so that they cannot be analyzed (I marked them as 'skip').

Now, lets review some pefile analysis, and at last, see if you can do some classification or clustering on this malware dataset.

### Download dump data #1


In [ ]:
# Here is the 1st dump json file.
!wget https://drive.google.com/uc?id=17TkMc-xXNyQ4muO9hmNGPrGb_mNfqlS9 -O worm_unpacked_dll_call.json

--2022-04-15 06:24:00--  https://drive.google.com/uc?id=17TkMc-xXNyQ4muO9hmNGPrGb_mNfqlS9
Resolving drive.google.com (drive.google.com)... 173.194.218.102, 173.194.218.101, 173.194.218.139, ...
Connecting to drive.google.com (drive.google.com)|173.194.218.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/oejq91avn8loogqhcunf9upua8h8rll1/1650003825000/17174434063503717705/*/17TkMc-xXNyQ4muO9hmNGPrGb_mNfqlS9 [following]
--2022-04-15 06:24:01--  https://doc-0c-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/oejq91avn8loogqhcunf9upua8h8rll1/1650003825000/17174434063503717705/*/17TkMc-xXNyQ4muO9hmNGPrGb_mNfqlS9
Resolving doc-0c-38-docs.googleusercontent.com (doc-0c-38-docs.googleusercontent.com)... 173.194.217.132, 2607:f8b0:400c:c13::84
Connecting to doc-0c-38-docs.googleusercontent.com (doc-0c-38-docs.googleusercontent.com)|173.194.2

In [ ]:
# The size should be 363255 bytes.
!ls -al

total 3160
drwxr-xr-x 1 root root    4096 Apr 15 06:24 .
drwxr-xr-x 1 root root    4096 Apr 15 03:49 ..
drwxr-xr-x 1 root root    4096 Apr  8 13:31 .config
drwxr-xr-x 1 root root    4096 Apr  8 13:32 sample_data
-rw-r--r-- 1 root root   12481 Apr 15 06:23 worm.csv
-rw-r--r-- 1 root root  363255 Apr 15 06:24 worm_unpacked_dll_call.json
-rw-r--r-- 1 root root 2838048 Apr 15 03:53 worm_unpacked_pe_dump.pickle


In [ ]:
# pefile information is stored in a json file.
# You can retrive it by using the following code.
# I convert the file to a python dictionary

import json

with open('worm_unpacked_dll_call.json') as json_file:
    worm_dict = json.load(json_file)
 
    # Print the type of data variable
    print("Type:", type(worm_dict))

Type: <class 'dict'>


In [ ]:
# the dictionary key is the index of the malware
# they are the MD5 value of the malware samples
worm_dict.keys()

dict_keys(['ffff5099125bceaea61251684fbdb770', 'fff80ae9338aec1e979ad1ddeb060560', 'fff000bb386ed5fca21866fab5172500', 'fff71cb8da44b471b448dad8d3c1ac20', 'ffe894389225f1d102769d89261acda0', 'fffda772e9682b5cd713a1aff6bebe71', 'fff1a3306e69df859b43e71cc2671f00', 'fffd904f5de5d9ef529b07b3bcf6f5b0', 'ffde11ff4926d17df917040683095b50', 'fff27dfe35219a2f7ab157fec9a6da40', 'ffe91510f165d479db6492d973ade310', 'ffdb3c00f7cc7c25f8240eed3a1d4a50', 'ffe189289e765a331b9b4f04fbfc7bb0', 'ffe4dabd0d2e8075aee4e7db15ce0d80', 'ffe5eaa82debc744b9c9c900fa22bc70', 'ffede07462c60920e4d59e84b65b7030', 'ffe268ffc185abe5b68970e45eb74810', 'fff2e49743064367fc7612168d185480', 'ffe19ece6b385773c362198940f76460', 'fff3236a1ded79b7c1138fca11971f60', 'fff1cc37ea691d75e793c642f6da3340', 'fff9d17e467f93121d15481266a63950', 'ffddcf15d798e8e22b6ec59232ab1fc0', 'ffeb34f7748ac1d42a9ea9e522040480', 'ffddeedc483f0ab09ddc5cb55d93f8f0', 'ffeeb351089c255e152dd2c79c55b250', 'fff26188530a82f3667c66da19168f20', 'ffff0fb5fed019a0

In [ ]:
# file #1 shows the dynamic library and function calls used by the malware.
worm_dict['ffff5099125bceaea61251684fbdb770']

{'advapi32.dll': ['RegCloseKey',
  'RegCreateKeyExA',
  'RegFlushKey',
  'RegOpenKeyExA',
  'RegQueryValueExA',
  'RegSetValueExA'],
 'kernel32.dll': ['CloseHandle',
  'CreateFileA',
  'CreateMutexA',
  'ExitProcess',
  'FreeLibrary',
  'GetCommandLineA',
  'GetCurrentThreadId',
  'GetDiskFreeSpaceA',
  'GetFileSize',
  'GetFileType',
  'GetLastError',
  'GetModuleFileNameA',
  'GetModuleHandleA',
  'GetProcAddress',
  'GetProcessHeap',
  'GetStdHandle',
  'GetSystemTime',
  'GetVersionExA',
  'HeapAlloc',
  'HeapFree',
  'HeapReAlloc',
  'LoadLibraryA',
  'LocalAlloc',
  'RaiseException',
  'ReadFile',
  'RtlUnwind',
  'SetEndOfFile',
  'SetFilePointer',
  'TlsGetValue',
  'TlsSetValue',
  'UnhandledExceptionFilter',
  'WaitForSingleObject',
  'WriteFile'],
 'user32.dll': ['CharNextA',
  'GetWindowLongA',
  'MessageBoxA',
  'SetWindowLongA',
  'ShowWindow']}

### Download dump data #2


In [ ]:
# Here is the 2nd dump pickle file.
!wget https://drive.google.com/uc?id=1wLdkVEUsblApCaV2IJbu7XSP6agVUz7- -O worm_unpacked_pe_dump.pickle

--2022-04-15 06:24:15--  https://drive.google.com/uc?id=1wLdkVEUsblApCaV2IJbu7XSP6agVUz7-
Resolving drive.google.com (drive.google.com)... 173.194.218.100, 173.194.218.101, 173.194.218.113, ...
Connecting to drive.google.com (drive.google.com)|173.194.218.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/pg5236fnsp4ds44p0rd2inmq2mc80epv/1650003825000/17174434063503717705/*/1wLdkVEUsblApCaV2IJbu7XSP6agVUz7- [following]
--2022-04-15 06:24:16--  https://doc-0c-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/pg5236fnsp4ds44p0rd2inmq2mc80epv/1650003825000/17174434063503717705/*/1wLdkVEUsblApCaV2IJbu7XSP6agVUz7-
Resolving doc-0c-38-docs.googleusercontent.com (doc-0c-38-docs.googleusercontent.com)... 173.194.217.132, 2607:f8b0:400c:c13::84
Connecting to doc-0c-38-docs.googleusercontent.com (doc-0c-38-docs.googleusercontent.com)|173.194.2

In [ ]:
!ls -al

total 3160
drwxr-xr-x 1 root root    4096 Apr 15 06:24 .
drwxr-xr-x 1 root root    4096 Apr 15 03:49 ..
drwxr-xr-x 1 root root    4096 Apr  8 13:31 .config
drwxr-xr-x 1 root root    4096 Apr  8 13:32 sample_data
-rw-r--r-- 1 root root   12481 Apr 15 06:23 worm.csv
-rw-r--r-- 1 root root  363255 Apr 15 06:24 worm_unpacked_dll_call.json
-rw-r--r-- 1 root root 2838048 Apr 15 06:24 worm_unpacked_pe_dump.pickle


In [ ]:
import pickle
print(pickle.HIGHEST_PROTOCOL) # should be 4

4


In [ ]:
with open('worm_unpacked_pe_dump.pickle', 'rb') as f:
    m_dict = pickle.load(f)
print(m_dict.keys())

dict_keys(['ffff5099125bceaea61251684fbdb770', 'fff80ae9338aec1e979ad1ddeb060560', 'fff000bb386ed5fca21866fab5172500', 'fff71cb8da44b471b448dad8d3c1ac20', 'ffe894389225f1d102769d89261acda0', 'fffda772e9682b5cd713a1aff6bebe71', 'fff1a3306e69df859b43e71cc2671f00', 'fffd904f5de5d9ef529b07b3bcf6f5b0', 'ffde11ff4926d17df917040683095b50', 'fff27dfe35219a2f7ab157fec9a6da40', 'ffe91510f165d479db6492d973ade310', 'ffdb3c00f7cc7c25f8240eed3a1d4a50', 'ffe189289e765a331b9b4f04fbfc7bb0', 'ffe4dabd0d2e8075aee4e7db15ce0d80', 'ffe5eaa82debc744b9c9c900fa22bc70', 'ffede07462c60920e4d59e84b65b7030', 'ffe268ffc185abe5b68970e45eb74810', 'fff2e49743064367fc7612168d185480', 'ffe19ece6b385773c362198940f76460', 'fff3236a1ded79b7c1138fca11971f60', 'fff1cc37ea691d75e793c642f6da3340', 'fff9d17e467f93121d15481266a63950', 'ffddcf15d798e8e22b6ec59232ab1fc0', 'ffeb34f7748ac1d42a9ea9e522040480', 'ffddeedc483f0ab09ddc5cb55d93f8f0', 'ffeeb351089c255e152dd2c79c55b250', 'fff26188530a82f3667c66da19168f20', 'ffff0fb5fed019a0

In [ ]:
# the detail info a malware in the dictionary 
m_dict['ffff5099125bceaea61251684fbdb770']

{'DOS_HEADER': {'Structure': 'IMAGE_DOS_HEADER',
  'e_cblp': {'FileOffset': 2, 'Offset': 2, 'Value': 80},
  'e_cp': {'FileOffset': 4, 'Offset': 4, 'Value': 2},
  'e_cparhdr': {'FileOffset': 8, 'Offset': 8, 'Value': 4},
  'e_crlc': {'FileOffset': 6, 'Offset': 6, 'Value': 0},
  'e_cs': {'FileOffset': 22, 'Offset': 22, 'Value': 0},
  'e_csum': {'FileOffset': 18, 'Offset': 18, 'Value': 0},
  'e_ip': {'FileOffset': 20, 'Offset': 20, 'Value': 0},
  'e_lfanew': {'FileOffset': 60, 'Offset': 60, 'Value': 256},
  'e_lfarlc': {'FileOffset': 24, 'Offset': 24, 'Value': 64},
  'e_magic': {'FileOffset': 0, 'Offset': 0, 'Value': 23117},
  'e_maxalloc': {'FileOffset': 12, 'Offset': 12, 'Value': 65535},
  'e_minalloc': {'FileOffset': 10, 'Offset': 10, 'Value': 15},
  'e_oemid': {'FileOffset': 36, 'Offset': 36, 'Value': 0},
  'e_oeminfo': {'FileOffset': 38, 'Offset': 38, 'Value': 0},
  'e_ovno': {'FileOffset': 26, 'Offset': 26, 'Value': 26},
  'e_res': {'FileOffset': 28,
   'Offset': 28,
   'Value': '\\x

### A dump file example

The above #2 pickle file are generated from pe's dump_dict() funciton.

In [ ]:
# Get a 32-bit putty pe file
!wget https://the.earth.li/~sgtatham/putty/latest/w32/putty.exe -O putty32.exe

--2022-04-15 06:24:32--  https://the.earth.li/~sgtatham/putty/latest/w32/putty.exe
Resolving the.earth.li (the.earth.li)... 93.93.131.124, 2a00:1098:86:4d:c0ff:ee:15:900d
Connecting to the.earth.li (the.earth.li)|93.93.131.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://the.earth.li/~sgtatham/putty/0.76/w32/putty.exe [following]
--2022-04-15 06:24:33--  https://the.earth.li/~sgtatham/putty/0.76/w32/putty.exe
Reusing existing connection to the.earth.li:443.
HTTP request sent, awaiting response... 200 OK
Length: 1180904 (1.1M) [application/x-msdos-program]
Saving to: ‘putty32.exe’

putty32.exe         100%[===================>]   1.13M  2.11MB/s    in 0.5s    

2022-04-15 06:24:34 (2.11 MB/s) - ‘putty32.exe’ saved [1180904/1180904]



In [ ]:
!pip install pefile

     |████████████████████████████████| 72 kB 453 kB/s 
  Created wheel for pefile: filename=pefile-2021.9.3-py3-none-any.whl size=68844 sha256=3cca012370c5a97b2d13be60ae34f0266d66bd9fe4e12acaa7e06c5612b05070
  Stored in directory: /root/.cache/pip/wheels/50/be/9b/d19a6151b9c6e303f823faedf03193d7e90dac4009a0bdafeb
Successfully built pefile


In [ ]:
import pefile

pef = pefile.PE("/content/putty32.exe", True)
dump_dict = pef.dump_dict()

In [ ]:
# a pefile dump dictionary has some information
# they are stored in different "keys"
print(dump_dict.keys())

dict_keys(['DOS_HEADER', 'NT_HEADERS', 'FILE_HEADER', 'Flags', 'OPTIONAL_HEADER', 'DllCharacteristics', 'PE Sections', 'Directories', 'Version Information', 'Imported symbols', 'Resource directory', 'LOAD_CONFIG', 'Base relocations'])


In [ ]:
# show one of the data in 'DOS_HEADER' key
# and it looks like a python dictionary
# you can put this data in a json or a dictionaary structure for further analysis
print(dump_dict['DOS_HEADER'])

{'Structure': 'IMAGE_DOS_HEADER', 'e_magic': {'FileOffset': 0, 'Offset': 0, 'Value': 23117}, 'e_cblp': {'FileOffset': 2, 'Offset': 2, 'Value': 120}, 'e_cp': {'FileOffset': 4, 'Offset': 4, 'Value': 1}, 'e_crlc': {'FileOffset': 6, 'Offset': 6, 'Value': 0}, 'e_cparhdr': {'FileOffset': 8, 'Offset': 8, 'Value': 4}, 'e_minalloc': {'FileOffset': 10, 'Offset': 10, 'Value': 0}, 'e_maxalloc': {'FileOffset': 12, 'Offset': 12, 'Value': 0}, 'e_ss': {'FileOffset': 14, 'Offset': 14, 'Value': 0}, 'e_sp': {'FileOffset': 16, 'Offset': 16, 'Value': 0}, 'e_csum': {'FileOffset': 18, 'Offset': 18, 'Value': 0}, 'e_ip': {'FileOffset': 20, 'Offset': 20, 'Value': 0}, 'e_cs': {'FileOffset': 22, 'Offset': 22, 'Value': 0}, 'e_lfarlc': {'FileOffset': 24, 'Offset': 24, 'Value': 64}, 'e_ovno': {'FileOffset': 26, 'Offset': 26, 'Value': 0}, 'e_res': {'FileOffset': 28, 'Offset': 28, 'Value': '\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00'}, 'e_oemid': {'FileOffset': 36, 'Offset': 36, 'Value': 0}, 'e_oeminfo': {'FileOffset': 

In [ ]:
# for eample, I put it in a dictionaty
# now I can easily retrive the info stored in it by keys
dos_header_dict = dict(dump_dict['DOS_HEADER'])
print(dos_header_dict.keys())

print(dos_header_dict['Structure'])  # name
print(dos_header_dict['e_magic'])  # e_magic

dict_keys(['Structure', 'e_magic', 'e_cblp', 'e_cp', 'e_crlc', 'e_cparhdr', 'e_minalloc', 'e_maxalloc', 'e_ss', 'e_sp', 'e_csum', 'e_ip', 'e_cs', 'e_lfarlc', 'e_ovno', 'e_res', 'e_oemid', 'e_oeminfo', 'e_res2', 'e_lfanew'])
IMAGE_DOS_HEADER
{'FileOffset': 0, 'Offset': 0, 'Value': 23117}


In [ ]:
# another example

file_header_dict = dict(dump_dict['FILE_HEADER'])
print(file_header_dict.keys())

print(file_header_dict['Machine'])
print(file_header_dict['NumberOfSections'])
print(file_header_dict['TimeDateStamp'])

dict_keys(['Structure', 'Machine', 'NumberOfSections', 'TimeDateStamp', 'PointerToSymbolTable', 'NumberOfSymbols', 'SizeOfOptionalHeader', 'Characteristics'])
{'FileOffset': 124, 'Offset': 0, 'Value': 332}
{'FileOffset': 126, 'Offset': 2, 'Value': 6}
{'FileOffset': 128, 'Offset': 4, 'Value': '0x60E96DBB [Sat Jul 10 09:51:55 2021 UTC]'}


In [ ]:
# You can even print all dump_dict to find some valueable infomation.
# pprint stands for pretty print, which makes you easier to view the data
import pprint
pprint.pprint(dump_dict['Imported symbols']) # change this to other keys

[[{'Characteristics': {'FileOffset': 774408, 'Offset': 0, 'Value': 777660},
   'FirstThunk': {'FileOffset': 774424, 'Offset': 16, 'Value': 778996},
   'ForwarderChain': {'FileOffset': 774416, 'Offset': 8, 'Value': 0},
   'Name': {'FileOffset': 774420, 'Offset': 12, 'Value': 786064},
   'OriginalFirstThunk': {'FileOffset': 774408, 'Offset': 0, 'Value': 777660},
   'Structure': 'IMAGE_IMPORT_DESCRIPTOR',
   'TimeDateStamp': {'FileOffset': 774412,
                     'Offset': 4,
                     'Value': '0x0        [Thu Jan  1 00:00:00 1970 UTC]'}},
  {'DLL': b'GDI32.dll', 'Hint': 41, 'Name': b'CreateBitmap'},
  {'DLL': b'GDI32.dll', 'Hint': 48, 'Name': b'CreateCompatibleBitmap'},
  {'DLL': b'GDI32.dll', 'Hint': 49, 'Name': b'CreateCompatibleDC'},
  {'DLL': b'GDI32.dll', 'Hint': 63, 'Name': b'CreateFontA'},
  {'DLL': b'GDI32.dll', 'Hint': 64, 'Name': b'CreateFontIndirectA'},
  {'DLL': b'GDI32.dll', 'Hint': 77, 'Name': b'CreatePalette'},
  {'DLL': b'GDI32.dll', 'Hint': 79, 'Name': b

### Analyze the downloaded data

Now you can analyze the 242 malware samples by #1 and/or #2 dump files.

In [ ]:
# The id of the malwares are MD5 in the worm.csv.
# and/or the keys of worm_dict and m_dict.
# Note that you should skip some samples specified in worm.csv.

for key in m_dict.keys():
  print(key)

ffff5099125bceaea61251684fbdb770
fff80ae9338aec1e979ad1ddeb060560
fff000bb386ed5fca21866fab5172500
fff71cb8da44b471b448dad8d3c1ac20
ffe894389225f1d102769d89261acda0
fffda772e9682b5cd713a1aff6bebe71
fff1a3306e69df859b43e71cc2671f00
fffd904f5de5d9ef529b07b3bcf6f5b0
ffde11ff4926d17df917040683095b50
fff27dfe35219a2f7ab157fec9a6da40
ffe91510f165d479db6492d973ade310
ffdb3c00f7cc7c25f8240eed3a1d4a50
ffe189289e765a331b9b4f04fbfc7bb0
ffe4dabd0d2e8075aee4e7db15ce0d80
ffe5eaa82debc744b9c9c900fa22bc70
ffede07462c60920e4d59e84b65b7030
ffe268ffc185abe5b68970e45eb74810
fff2e49743064367fc7612168d185480
ffe19ece6b385773c362198940f76460
fff3236a1ded79b7c1138fca11971f60
fff1cc37ea691d75e793c642f6da3340
fff9d17e467f93121d15481266a63950
ffddcf15d798e8e22b6ec59232ab1fc0
ffeb34f7748ac1d42a9ea9e522040480
ffddeedc483f0ab09ddc5cb55d93f8f0
ffeeb351089c255e152dd2c79c55b250
fff26188530a82f3667c66da19168f20
ffff0fb5fed019a0cdf8c0fc31c40260
ffdf44585f9e8a8e709e5ebe97d4d790
ffdb2677884ac3b6feb1b436747b5f70
fff43e34a6

In [ ]:
# so the fist malware m_dict is
first_dict = m_dict['fff23e2c4f99371e5007aa18a6476120']

# you can now access the data you want
# for example print all sections and their info.
for sec in first_dict['PE Sections']:
  print(sec)

# or like this (as what we see in pefile demonstration)
print("\nName, Misc_VirtualSize, SizeOfRawData, Entropy, MD5")
for sec in first_dict['PE Sections']:
  print(sec['Name']['Value'], sec['Misc_VirtualSize']['Value'], sec['SizeOfRawData']['Value'], sec['Entropy'], sec['MD5'])

{'Structure': 'IMAGE_SECTION_HEADER', 'Name': {'FileOffset': 480, 'Offset': 0, 'Value': '.text\\x00\\x00\\x00'}, 'Misc': {'FileOffset': 488, 'Offset': 8, 'Value': 510}, 'Misc_PhysicalAddress': {'FileOffset': 488, 'Offset': 8, 'Value': 510}, 'Misc_VirtualSize': {'FileOffset': 488, 'Offset': 8, 'Value': 510}, 'VirtualAddress': {'FileOffset': 492, 'Offset': 12, 'Value': 4096}, 'SizeOfRawData': {'FileOffset': 496, 'Offset': 16, 'Value': 512}, 'PointerToRawData': {'FileOffset': 500, 'Offset': 20, 'Value': 1024}, 'PointerToRelocations': {'FileOffset': 504, 'Offset': 24, 'Value': 0}, 'PointerToLinenumbers': {'FileOffset': 508, 'Offset': 28, 'Value': 0}, 'NumberOfRelocations': {'FileOffset': 512, 'Offset': 32, 'Value': 0}, 'NumberOfLinenumbers': {'FileOffset': 514, 'Offset': 34, 'Value': 0}, 'Characteristics': {'FileOffset': 516, 'Offset': 36, 'Value': 1610612768}, 'Flags': ['IMAGE_SCN_CNT_CODE', 'IMAGE_SCN_MEM_EXECUTE', 'IMAGE_SCN_MEM_READ'], 'Entropy': 5.676232928844902, 'MD5': 'e15dfbe2eab6

In [ ]:
# Here you can output the dll and windows api used by a pe file
# You may print dump_dict['Imported symbols'] first
# and try to locate the info you want to retrive.

api_dict = dict()
for records in first_dict['Imported symbols']:
  for record in records:
    if 'Name' in record:
      if isinstance(record['Name'], bytes):
        dll = record['DLL'].decode("ascii")
        name = record['Name'].decode("ascii")
        if dll not in api_dict:
          api_dict[dll] = set()
        api_dict[dll].add(name)

for dll in api_dict:
  print(dll)
  print("", api_dict[dll])

KERNEL32.dll
 {'LoadLibraryW', 'CreateFileW', 'VirtualAlloc', 'GetProcAddress', 'ReadFile'}
MSVCRT.dll
 {'malloc', '_initterm', 'free', '_adjust_fdiv'}


### Extract more information

Now its your turn.
Try to buld a feature table for these 242 malwares.

Hint : these 242 malware samples are from several families. Can you classify or cluster them into groups (w.r.t. family)? You can do supervised learning or unsupervised learning.

I suggest to clean the labels made by Microsoft in the csv file first.

Then, the first try might be creating clusers or perform classificaiton based on the libraries and functions used by the samples, and see if you can create a 'distance function' to known the distance between samples (w.r.t. function calls used). Now, since your have distance, then ML algorithms can be applied.

### **Your solution**

In [ ]:
new_df=_df.drop(['skip'],axis=1)
new_df

,MD5,Microsoft,noFamilyName,UPXpacked
0,ffdf141aded498e641b45b0e2188e360,Backdoor:Win32/Berbew.DR,0.0,0
1,fff738fcbe3244b4048edb3e289dd450,Backdoor:Win32/Berbew.DR,0.0,0
2,ffe268ffc185abe5b68970e45eb74810,Backdoor:Win32/Berbew.I,0.0,0
3,fff6a4b8265ea0a0d778697c4096c230,Backdoor:Win32/Delf,0.0,0
4,fff1517aebae3c980f13efd7edea2480,Backdoor:Win32/Delf.DU,0.0,0
...,...,...,...,...
237,fffed384aeba28e1474467058b4756a0,NaN,NaN,0
238,ffff2a37e92c2ae42c5b8230c1c0c8e0,NaN,NaN,1
239,ffff31ad7e82b8cfe50f5ba21e13a450,NaN,NaN,0
240,ffff90de328dc28095d7610c18b20130,NaN,NaN,0


In [ ]:
new_df=new_df[(new_df.UPXpacked!=1)&(new_df.noFamilyName!=1)]
new_df

,MD5,Microsoft,noFamilyName,UPXpacked
0,ffdf141aded498e641b45b0e2188e360,Backdoor:Win32/Berbew.DR,0.0,0
1,fff738fcbe3244b4048edb3e289dd450,Backdoor:Win32/Berbew.DR,0.0,0
2,ffe268ffc185abe5b68970e45eb74810,Backdoor:Win32/Berbew.I,0.0,0
3,fff6a4b8265ea0a0d778697c4096c230,Backdoor:Win32/Delf,0.0,0
4,fff1517aebae3c980f13efd7edea2480,Backdoor:Win32/Delf.DU,0.0,0
...,...,...,...,...
236,fffebd5a2d4f2072665a3592e44a8600,NaN,NaN,0
237,fffed384aeba28e1474467058b4756a0,NaN,NaN,0
239,ffff31ad7e82b8cfe50f5ba21e13a450,NaN,NaN,0
240,ffff90de328dc28095d7610c18b20130,NaN,NaN,0


In [ ]:
import pandas as pd
dump1_df=pd.read_json('worm_unpacked_dll_call.json')
dump1_df=dump1_df.T
dump1_df

,kernel32.dll,user32.dll,advapi32.dll,KERNEL32.DLL,oleaut32.dll,mpr.dll,shell32.dll,wsock32.dll,wininet.dll,KERNEL32.dll,...,newdev.dll,ntdll.dll,Setupapi.dll,OPENGL32.dll,Msacm32.dll,crypt32.dll,pstorec.dll,rasapi32.dll,MMCNDMGR.dll,CRYPTUI.dll
ffff5099125bceaea61251684fbdb770,"[CloseHandle, CreateFileA, CreateMutexA, ExitP...","[CharNextA, GetWindowLongA, MessageBoxA, SetWi...","[RegCloseKey, RegCreateKeyExA, RegFlushKey, Re...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fff80ae9338aec1e979ad1ddeb060560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fff000bb386ed5fca21866fab5172500,NaN,"[CharNextA, GetKeyboardType, GetSystemMetrics,...","[RegCloseKey, RegCreateKeyExA, RegFlushKey, Re...","[CloseHandle, CompareStringA, CopyFileA, Creat...","[SafeArrayCreate, SafeArrayGetElement, SafeArr...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fff71cb8da44b471b448dad8d3c1ac20,NaN,"[CharNextA, GetKeyboardType, GetSystemMetrics,...","[RegCloseKey, RegOpenKeyA, RegOpenKeyExA, RegQ...","[CloseHandle, CreateDirectoryA, CreateFileA, D...",[SysFreeString],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ffe894389225f1d102769d89261acda0,NaN,"[CharNextA, GetKeyboardType, GetSystemMetrics,...","[RegCloseKey, RegOpenKeyA, RegOpenKeyExA, RegQ...","[CloseHandle, CreateDirectoryA, CreateFileA, D...",[SysFreeString],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffe68bc360d090ab1cf3a3e02bd9a9b0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[AddAtomA, ExitProcess, FindAtomA, FreeConsole...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ffe5c541f680e4833789b825d25b4410,NaN,"[ActivateKeyboardLayout, AdjustWindowRectEx, B...","[RegCloseKey, RegConnectRegistryW, RegCreateKe...","[CloseHandle, CompareStringW, CreateDirectoryW...","[CreateErrorInfo, DispGetIDsOfNames, GetActive...",NaN,"[SHBrowseForFolderW, SHGetDesktopFolder, SHGet...","[WSAAsyncGetHostByName, WSACancelAsyncRequest,...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fffdf07a3139728ce4a73b6d540fd9c0,NaN,"[CharNextA, GetKeyboardType, GetSystemMetrics,...","[RegCloseKey, RegCreateKeyExA, RegFlushKey, Re...","[CloseHandle, CompareStringA, CopyFileA, Creat...","[SafeArrayCreate, SafeArrayGetElement, SafeArr...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ffe8754077406068ea41fd5f3ad3d200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#drop the rows that all the vlaues are NAN
dump1_df=dump1_df.dropna(axis=0,how='all')
dump1_df

,kernel32.dll,user32.dll,advapi32.dll,KERNEL32.DLL,oleaut32.dll,mpr.dll,shell32.dll,wsock32.dll,wininet.dll,KERNEL32.dll,...,newdev.dll,ntdll.dll,Setupapi.dll,OPENGL32.dll,Msacm32.dll,crypt32.dll,pstorec.dll,rasapi32.dll,MMCNDMGR.dll,CRYPTUI.dll
ffff5099125bceaea61251684fbdb770,"[CloseHandle, CreateFileA, CreateMutexA, ExitP...","[CharNextA, GetWindowLongA, MessageBoxA, SetWi...","[RegCloseKey, RegCreateKeyExA, RegFlushKey, Re...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fff000bb386ed5fca21866fab5172500,NaN,"[CharNextA, GetKeyboardType, GetSystemMetrics,...","[RegCloseKey, RegCreateKeyExA, RegFlushKey, Re...","[CloseHandle, CompareStringA, CopyFileA, Creat...","[SafeArrayCreate, SafeArrayGetElement, SafeArr...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fff71cb8da44b471b448dad8d3c1ac20,NaN,"[CharNextA, GetKeyboardType, GetSystemMetrics,...","[RegCloseKey, RegOpenKeyA, RegOpenKeyExA, RegQ...","[CloseHandle, CreateDirectoryA, CreateFileA, D...",[SysFreeString],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ffe894389225f1d102769d89261acda0,NaN,"[CharNextA, GetKeyboardType, GetSystemMetrics,...","[RegCloseKey, RegOpenKeyA, RegOpenKeyExA, RegQ...","[CloseHandle, CreateDirectoryA, CreateFileA, D...",[SysFreeString],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fffda772e9682b5cd713a1aff6bebe71,"[CloseHandle, CopyFileA, CreateFileA, CreateMu...","[CharNextA, DispatchMessageA, GetKeyboardType,...","[RegCloseKey, RegOpenKeyA, RegOpenKeyExA, RegQ...",NaN,[SysFreeString],"[WNetAddConnection2A, WNetCancelConnectionA]",[ShellExecuteA],"[closesocket, connect, htons, inet_addr, socket]",[InternetGetConnectedState],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffed8749b0068fe82e48b34fecc07a4a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[CancelIo, CopyFileA, DeleteFileA, GetCommandL...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[DllCanUnloadNow, DllGetClassObject, DllRegist...","[CryptUIDlgViewContext, CryptUIStartCertMgr, C..."
ffe68bc360d090ab1cf3a3e02bd9a9b0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[AddAtomA, ExitProcess, FindAtomA, FreeConsole...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ffe5c541f680e4833789b825d25b4410,NaN,"[ActivateKeyboardLayout, AdjustWindowRectEx, B...","[RegCloseKey, RegConnectRegistryW, RegCreateKe...","[CloseHandle, CompareStringW, CreateDirectoryW...","[CreateErrorInfo, DispGetIDsOfNames, GetActive...",NaN,"[SHBrowseForFolderW, SHGetDesktopFolder, SHGet...","[WSAAsyncGetHostByName, WSACancelAsyncRequest,...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fffdf07a3139728ce4a73b6d540fd9c0,NaN,"[CharNextA, GetKeyboardType, GetSystemMetrics,...","[RegCloseKey, RegCreateKeyExA, RegFlushKey, Re...","[CloseHandle, CompareStringA, CopyFileA, Creat...","[SafeArrayCreate, SafeArrayGetElement, SafeArr...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
!pip install gower

  Created wheel for gower: filename=gower-0.0.5-py3-none-any.whl size=4232 sha256=b3ed57a65207f454ad6a87c289c1d28f587bf58a6354b560ad17c0d2341e9cae
  Stored in directory: /root/.cache/pip/wheels/3e/f9/9a/67122a959a424e9cbb4557a8366c871a30e31cd75f0d003db4
Successfully built gower


In [ ]:
import gower

distance_matrix = gower.gower_matrix(dump1_df)
distance_matrix

array([[0.9589041 , 0.98630136, 1.        , ..., 1.        , 0.98630136,
        0.9589041 ],
       [0.98630136, 0.94520545, 0.98630136, ..., 1.        , 0.94520545,
        0.98630136],
       [1.        , 0.98630136, 0.94520545, ..., 1.        , 0.98630136,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 0.8082192 , 1.        ,
        1.        ],
       [0.98630136, 0.94520545, 0.98630136, ..., 1.        , 0.94520545,
        0.98630136],
       [0.9589041 , 0.98630136, 1.        , ..., 1.        , 0.98630136,
        0.9589041 ]], dtype=float32)

In [ ]:
#Using DBSCAN to cluster
from sklearn.cluster import DBSCAN

# Configuring the parameters of the clustering algorithm
dbscan_cluster = DBSCAN(eps=0.2, min_samples=10)

# Fitting the clustering algorithm
dbscan_cluster.fit(distance_matrix)

# Adding the results to a new column in the dataframe
dump1_df['cluster'] = dbscan_cluster.labels_
dump1_df['cluster']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


ffff5099125bceaea61251684fbdb770    0
fff000bb386ed5fca21866fab5172500    1
fff71cb8da44b471b448dad8d3c1ac20    2
ffe894389225f1d102769d89261acda0    2
fffda772e9682b5cd713a1aff6bebe71    2
                                   ..
ffed8749b0068fe82e48b34fecc07a4a    2
ffe68bc360d090ab1cf3a3e02bd9a9b0    2
ffe5c541f680e4833789b825d25b4410   -1
fffdf07a3139728ce4a73b6d540fd9c0    1
fff02d4e75d85a78af4698430e3390d0    0
Name: cluster, Length: 198, dtype: int64